<a href="https://colab.research.google.com/github/avrymi-asraf/AML/blob/main/Ex3/Ex3-interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and tools

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.functional import pad

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import re
import math
from tqdm import tqdm, tqdm_pandas, trange

In [2]:
from IPython.display import Markdown, clear_output, display

md = lambda x: display(Markdown(x))
he_md = lambda x: display(Markdown(f'<div dir="rtl" lang="he" xml:lang="he">{x}</div>'))

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
# for colab
!mkdir src

!wget -O /content/src/data_load.py https://raw.githubusercontent.com/avrymi-asraf/AML/main/Ex3/src/data_load.py
!wget -O /content/src/models.py https://raw.githubusercontent.com/avrymi-asraf/AML/main/Ex3/src/models.py
!wget -O /content/src/vicreg_objectives.py https://raw.githubusercontent.com/avrymi-asraf/AML/main/Ex3/src/vicreg_objectives.py
clear_output()

In [5]:
# for local machine
from src.models import Encoder, Projector
from src.vicreg_objectives import vicreg_loss_detailed, vicreg_loss_performance
from src.data_load import (
    load_cifar10,
    load_mnist,
    load_combined_test_set,
    load_vicreg_cifar10,
    test,
)

# Q1: Training
Train VICReg on the CIFAR10 dataset. Plot the values of each of the 3 objectives (in separate
figures) as a function of the training batches. In your figures also include the loss terms values on the test set,computed once every epoch.

In [6]:
class VICeg(nn.Module):
    def __init__(self, en_dim=128, prj_dim=512):
        super(VICeg, self).__init__()
        self.encoder = Encoder(D=en_dim, device=DEVICE)
        self.projector = Projector(D=en_dim, proj_dim=prj_dim)

    def forward(self, x):
        return self.projector(self.encoder(x))

In [12]:
model = VICeg().to(DEVICE)


batch_size = 256
lr = 3 * 1e-3
optimizer = torch.optim.Adam(
    model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=10e-6
)
epochs = 20
loss_function = lambda z1, z2: vicreg_loss_performance(z1, z2)
loss_function_d = lambda z1, z2: vicreg_loss_detailed(z1, z2)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.



In [13]:
recorder = pd.DataFrame(
    columns=[
        "epoch",
        "epoch_loss",
        "inv_loss",
        "var_loss",
        "cov_loss",
        "test_loss",
    ],
    index=range(epochs),
)

train_loader, test_loader = load_vicreg_cifar10(batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
for epoch in range(epochs):
    model.train()
    epoch_loss = 1
    for i, (x1, x2) in tqdm(enumerate(train_loader)):
        x1, x2 = x1.to(DEVICE), x2.to(DEVICE)
        optimizer.zero_grad()
        z1, z2 = model(x1), model(x2)
        loss = loss_function(z1, z2)
        loss.backward()
        epoch_loss = loss.item() * 0.3 + epoch_loss * 0.7
        optimizer.step()

    model.eval()
    with torch.no_grad():
        inv_loss, var_loss, cov_loss, test_loss = 0, 0, 0, 0
        for i, (x1,_) in enumerate(test_loader):
            x1, x2 = x1.to(DEVICE), x1.clone().to(DEVICE)
            z1, z2 = model(x1), model(x2)
            _, loss_components = loss_function_d(z1, z2)
            inv_loss += loss_components["inv_loss"]
            var_loss += loss_components["var_loss"]
            cov_loss += loss_components["cov_loss"]
            test_loss += loss_components["loss"]

        loss, compomment = loss_function_d(z1, z2)
        recorder.loc[epoch] = {
            "epoch": epoch,
            "epoch_loss": epoch_loss,
            "inv_loss": inv_loss,
            "var_loss": var_loss,
            "cov_loss": cov_loss,
            "test_loss": test_loss,
        }
        clear_output(wait=True)
        px.line(
            recorder,
            y=["epoch_loss", "inv_loss", "var_loss", "cov_loss", "test_loss"],
        ).show()

In [17]:
torch.save(model.state_dict(), "vicreg_20_run.txt")

# Q2: PCA vs. T-SNE Visualizations.
Compute the representations of each test image using your trained encoder.
Map (using the sklearn library) the representation to a 2D space using: (i) PCA (ii) T-SNE. Plot the T-SNE and the PCA 2D representations, colored by their classes. Look at both visualizations (PCA vs. T-SNE), which one seems more effective for visualizations to you? Look at the T-SNE visualization. Did VICReg managed to capture the class information accurately? Which classes seem entangled to you? Explain.


In [ ]:
model.eval()

# Get the test data representations
test_representations = []
test_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(DEVICE)
        representations = model.encoder(images)
        test_representations.append(representations.cpu().numpy())
        test_labels.append(labels.cpu().numpy())

test_representations = np.concatenate(test_representations)
test_labels = np.concatenate(test_labels)


In [ ]:

# Perform PCA
pca = PCA(n_components=2)
pca_representations = pca.fit_transform(test_representations)

# Perform t-SNE
tsne = TSNE(n_components=2, perplexity=30)
tsne_representations = tsne.fit_transform(test_representations)

# Create a DataFrame for Plotly
pca_df = pd.DataFrame(
    {
        'PCA Component 1': pca_representations[:, 0],
        'PCA Component 2': pca_representations[:, 1],
        'Class': test_labels
    }
)

tsne_df = pd.DataFrame(
    {
        't-SNE Component 1': tsne_representations[:, 0],
        't-SNE Component 2': tsne_representations[:, 1],
        'Class': test_labels
    }
)


In [ ]:

# Plot PCA results with Plotly
fig_pca = px.scatter(
    pca_df, x='PCA Component 1', y='PCA Component 2',
    color='Class', title='PCA Visualization of Test Image Representations'
)
fig_pca.show()

# Plot t-SNE results with Plotly
fig_tsne = px.scatter(
    tsne_df, x='t-SNE Component 1', y='t-SNE Component 2',
    color='Class', title='t-SNE Visualization of Test Image Representations'
)
fig_tsne.show()

# Q3: Linear Probing.
Perform a linear probing (single FC layer) to the encoder’s representation. Train this
classifier on the representations of the CIFAR10 train set. Remember to freeze the encoder, i.e. do not update it. Compute the probing’s accuracy on the test set. What is the accuracy you reach with your classifier?
Note: classifier accuracy should be at least 60% on the test set.

# Q4: Ablation 1 - No Variance Term.
Modify the optimized objective, by removing the variance objective term
(µ = 0.). Using the representations from the modified encoder, perform the same PCA visualization from Q2, and the linear probing from Q3 (and include them in your report). Is the new accuracy better or worse? Can you see anything different in the representations visualization? Try to explain the difference in the accuracy using the visualizations.


# Q5: Ablation 2 - No Generated Neighbors.
Now, we would like to ablate VICReg by only removing the generated neighbors, using neighbors from the data itself:
First, compute the representations of your original VICReg, on all of the training set. In each step of training and for each image in the batch, use these representations to find the top 3 nearest neighbors, and randomly select 1 of them. Use the original image and this neighbor of it as your 2 views for the VICReg algorithm.

2 Practical Tips: (i) We find that training this algorithm for only a single epoch is more
beneficial. (ii) We recommend you to compute the neighboring indices of each image in advance, and delete the original VICReg model from your (GPU) memory. This will save both run time and GPU space.
Compute the linear probing accuracy, and report it. Is the accuracy different from the original linear probing from 3?
- If no, explain why do you think this change had no effect (what compensates the things that are missing?).
- If yes, explain what added value do you think the generated neighbors adds to the algorithm.

# Q6: Ablation 3 - Laplacian Eigenmaps.
After removing the generated neighbors, we would like to remove both it and the amortization at once. To do so, we will perform Laplacian Eigenmaps representation learning on the training data of CIFAR10. Since this algorithm is difficult to run, we ran it for you on 10K images (due to runtime limitations) and give you the T-SNE plotting of these representations in Fig. 2 3.
Compare this to VICReg’s T-SNE plot from Q2. Based on this visual and linear probing comparison, which method (VICReg vs. Laplacian Eigenmaps) seems more effective for downstream object classification?
Explain your answer in detail, including what do you think makes one algorithm to be more successful.


# Q7: Retrieval Evaluation
Now that we slowly “pealed” VICReg back to the laplacian eigenmaps algorithm,
we wish to evaluate it qualitatively. For the methods Q1 (VICReg) and Q5 (No Generated Neighbors) perform a
qualitative retrieval evaluation. That means:
- Select 10 random images from the training set, one from each class.
- For each selected image, use the representations of each of the evaluated methods, to find its 5 nearest neighbors
in the dataset.
- Plot the images together with their neighbors.
- Using the same images, perform the same visualization for the 5 most distant images in the dataset.

Using this visualization, explain what attributes each method attends to. What are the differences you see between
the different methods? Which one excels at keeping close images together? Which one excels at keeping distant
images far apart? Explain the differences between the methods in detail, as seen by this visualization. You may
select more than 1 image for a specific class if you wish to get a better understanding (Although it is not mandatory).

# Q1 - Anomaly Detection.
Using the CIFAR10 training data as reference for normal data, compute the kNN
density estimation for all the (CIFAR10 + MNIST) test set representations. Do this for both (i) VICReg (ii) VICReg
without generated neighbors. Use k = 2.


# Q2 - ROC AUC Evaluation
Plot the ROC Curve of both methods. Use the sklearn library for creating these
figures. In the title / legend incorporate the AUC of each method. Which method is ’better’? In a sentence or two,
explain why do you think its better.

# Q3 - Qualitative Evaluation - Ambiguity of Anomaly Detection.
Plot the 7 most anomalous samples according
to VICReg and VICReg without the generated neighbors, in two separate rows (you can split to different plots if
more convenient). Look at the results. Explain what aspects each method found to be anomalous. Keeping in mind
we did not give either of the methods any clues regarding which anomalies are we looking for, do you still think one
is better than the other? Explain your answer.

# Q1 - Clustering using K-Means
Use the implementation of sklearn to cluster the CIFAR10 training set to
10 clusters. Do this using the representations of VICReg once, and a second time for VICReg without generated
neighbors.

# Q2 - Visualizing the Clusters in 2D
Perform a dimensionality reduction to 2 dimensions using T-SNE, for
the representations of both methods (separately). For each method, plot the reduced embeddings in a 2D space twice
(side by side): (i) First, colored by their cluster index (according to the matching clustering from Q1). (ii) Second,
colored by their actual class index. In each figure, also plot the clusters centers in black color.
Look at the results, which method looks more successful at finding clusters? Which method looks more successful
at separating between the classes? Explain your answer, keeping in mind that similarly to Anomaly Detection there
are no clues for which clusters to look for.

# Q3 - Quantitative Analysis
Use the Silhouette Score on the clusterings of both methods (in their original embedding dimensions). Report the 2 different scores. Is this coherent with what you see in your visual analysis? Explain your answer